In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer

from torch.utils.data.dataset import Dataset

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/jerry/miniconda3/envs/agi/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda121_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /home/jerry/miniconda3/envs/agi/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda121_nocublaslt.so...


/home/jerry/miniconda3/envs/agi/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/jerry/miniconda3/envs/agi did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/jerry/miniconda3/envs/agi/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home/jerry/miniconda3/envs/agi/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [4]:
class MyDataset(Dataset):
    def __init__(self, tokenizer, file_path, block_size):
        super().__init__()

        self.tokenizer = tokenizer
        self.block_size = block_size

        with open(file_path, 'r') as f:
            lines = f.read().splitlines()

        # Split long lines into smaller parts
        lines = [line[i:i+block_size] for line in lines for i in range(0, len(line), block_size)]

        self.examples = [tokenizer.encode(line) for line in lines]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        input_ids = self.examples[i][:self.block_size]
        target_ids = self.examples[i][1:self.block_size+1]  # Shifted right

        # Pad input and target sequences to the block size
        input_ids = input_ids + [self.tokenizer.pad_token_id] * (self.block_size - len(input_ids))
        target_ids = target_ids + [-100] * (self.block_size - len(target_ids))  # -100 is the index that PyTorch uses to ignore a token in loss computation

        return {
            'input_ids': input_ids,
            'labels': target_ids,
        }


In [5]:
LORA_R = 8  # Some LoRA parameters
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
LEARNING_RATE = 2e-5
TARGET_MODULES = [
    'q_proj',
    'v_prol',
]

In [6]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias='none',
    task_type='CAUSAL_LM',
)

In [7]:
endpoint = "EleutherAI/gpt-j-6B"

In [8]:
# 1. Prepare your data
# Assume you have a 'text.txt' file containing your unlabeled data
model = AutoModelForCausalLM.from_pretrained(
    endpoint,
    load_in_8bit=True,
    device_map="auto"
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [9]:
model = prepare_model_for_int8_training(model)
model = get_peft_model(model, config)

In [10]:
# 2. Create a tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    endpoint,
    add_eos_token=True
)
tokenizer.pad_token_id = 0 # have pad_token_id different from eos token

In [11]:
# create dataset
dataset = MyDataset(
    tokenizer=tokenizer,
    file_path="./data/shakespere.txt",
    block_size=256,
)

In [12]:
# 5. Fine-tune the model
training_args = Seq2SeqTrainingArguments(
    output_dir="./gptl_finetuned/shakespear", # The output directory
    overwrite_output_dir=True, # Overwrite the content of the output directory
    learning_rate=LEARNING_RATE,
    num_train_epochs=2, # Number of training epochs
    per_device_train_batch_size=1, # Batch size for training
    save_steps=10000, # Number of updates steps before two checkpoint saves
    save_total_limit=2, # Limit the total amount of checkpoints and deletes the older checkpoints,
)





In [13]:
old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())
).__get__(model, type(model))

In [14]:
# import os

# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [15]:
# 3. Create a dataset
# data_dir = './data/txt'

# dataset = MyDataset(
#     tokenizer=tokenizer,
#     file_path=f'{data_dir}/all.txt',
#     block_size=512,
# )
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)    

In [16]:
trainer.train()

/home/jerry/miniconda3/envs/agi/lib/python3.11/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 111012912jerry. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/jerry/miniconda3/envs/agi/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
500,8.514500
1000,7.478100
1500,7.045700
2000,6.971000
2500,6.906300
3000,6.824400
3500,6.752800
4000,6.701500
4500,6.626200
5000,6.645300


/home/jerry/miniconda3/envs/agi/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/jerry/miniconda3/envs/agi/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/jerry/miniconda3/envs/agi/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/jerry/miniconda3/envs/agi/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: U

TrainOutput(global_step=65554, training_loss=6.159881325977289, metrics={'train_runtime': 149113.0529, 'train_samples_per_second': 0.44, 'train_steps_per_second': 0.44, 'total_flos': 5.88667390868226e+17, 'train_loss': 6.159881325977289, 'epoch': 2.0})

In [17]:
trainer.save_model()

In [18]:
from transformers import pipeline
from transformers import GenerationConfig

In [21]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Input:
{input}
### Response:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Response:
"""

In [ ]:
def generate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(temperature=0.2, top_p=0.75, num_beams=4),
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for seq in generation_output.sequences:
        output = tokenizer.decode(seq)
        print("Respuesta:", output.split("### Response:")[1].strip())

In [19]:
llm = pipeline('text2text-generation', model=model, tokenizer=tokenizer)
result = llm("please answer the question: what is hamlet")
print(result)

The model 'PeftModelForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/jerry/miniconda3/envs/agi/lib/python3.11/site-packages/torch/util

[{'generated_text': 'please answer the question: what is hamlet???.\n?.?.?.'}]


In [20]:
llm_new = pipeline('text2text-generation', model=model, tokenizer=tokenizer, device="cuda:0")
result_new = llm_new('please answer the question: to be or not to be')
print(result_new)

The model 'PeftModelForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'please answer the question: to be or not to be, is?\n\n. I,,'}]
